### MAHIR Imports

%pylab inline 
import os
import nibabel as nib
import numpy as np
from glob import glob
from scipy.ndimage import label

##### Load the text file with mse ids

subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
print subjects[:5]

##### define some paths to data

base_dir = "/data/henry7/PBR/subjects/"
template = "{base_dir}/{mseid}/lesions_manual/*/alignment_lesions.nii.gz"

## figure out a way to fill the template and glob for the file you need

lesion_file = #fill the template with the first subject, subjects[0]

## Load an image

img = nib.load(lesion_file)
data, affine = nib.get_data(), nib.get_affine()

## Number each lesion 

labels, nlabels = label(data==1)

## Get x,y,z coordinates for each lesion

np.nonzero

## Tranform coordinates to world space

np.dot([x,y,z,1], aff) #order might be switched 

In [355]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
from glob import glob
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


In [356]:
subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
print subjects[:5]

['mse1328' 'mse1334' 'mse1337' 'mse1343' 'mse1349']


In [357]:
import glob
import nibabel as nib

In [358]:
ms1328 = "/data/henry7/PBR/subjects/mse1328/lesions_manual/ms56-mse1328-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz"

In [359]:
for les_file_1328 in glob.glob("/data/henry7/PBR/subjects/mse1328/lesions_manual/*/alignment_lesions.nii.gz"):
    print les_file_1328
for seg_file_1328 in glob.glob("/data/henry7/PBR/subjects/mse1328/masks/*/segmentation.nii.gz"):
    print seg_file_1328

/data/henry7/PBR/subjects/mse1328/lesions_manual/ms56-mse1328-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1328/masks/ms56-mse1328-002-AX_T1_3D_IRSPGR/segmentation.nii.gz


In [360]:
les_img1328 = nib.load(les_file_1328)
les_img1328.dataobj
seg_img1328 = nib.load(seg_file_1328)
seg_img1328.dataobj

In [361]:
les_data1328 = les_img1328.get_data()
print les_data1328.shape
seg_data1328 = seg_img1328.get_data()
print seg_data1328.shape

(256, 256, 150)
(256, 256, 256)


In [362]:
les_affine1328 = les_img1328.get_affine()
print les_affine1328
seg_affine1328 = seg_img1328.get_affine()
print seg_affine1328
inv_seg_affine_1328 = np.linalg.inv(seg_affine1328)
print inv_seg_affine_1328

[[  -0.9375        0.            0.          123.6289978 ]
 [   0.            0.9375        0.          -93.36050415]
 [   0.            0.            1.          -44.42710114]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          131.6289978 ]
 [   0.            0.            1.         -101.36050415]
 [   0.           -1.            0.          173.57290649]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          131.6289978 ]
 [  -0.           -0.           -1.          173.57290649]
 [   0.            1.            0.          101.36050415]
 [   0.            0.            0.            1.        ]]


In [363]:
les_labels, nles_labels = label(les_data1328==[1])
seg_les_labels, seg_nles_labels = label(seg_data1328==[4])

In [364]:
nles_labels

11

In [365]:
seg_nles_labels

2

In [366]:
n = np.unique(les_labels)
print n

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [367]:
np.bincount(les_labels.ravel())

array([9829648,      69,     105,      54,      29,      69,      61,
            36,      47,     125,     111,      46])

In [368]:
def get_lesion_coord():
    all_les_coords = []
    for count in range(1, nles_labels+1):
        cur_les_coords = []
        x,y,z = np.nonzero(les_labels==count)
        for count2 in range(len(x)):
            cur_coord = [x[count2],y[count2],z[count2]]
            cur_les_coords.append(cur_coord)
        all_les_coords.append(cur_les_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_les_coords

In [369]:
def get_rs_coord(coordinates):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(les_affine1328, [coordinates[count][count2][0],
                                               coordinates[count][count2][1], 
                                               coordinates[count][count2][2],
                                               1])
            #rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

In [370]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine_1328, [coordinates[count][count2][0],
                                                    coordinates[count][count2][1], 
                                                    coordinates[count][count2][2],
                                                    1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

In [371]:
def average_func(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [int(average_x), int(average_y), int(average_z)]
        averages_all.append(averages)
        #print count, averages
    return averages_all

In [372]:
average_les = average_func(get_lesion_coord())
average_rs = average_func(get_rs_coord(get_lesion_coord()))
average_seg = average_func(get_seg_coord(get_rs_coord(get_lesion_coord())))

allcoord_les = get_lesion_coord()
allcoord_rs = get_rs_coord(get_lesion_coord())
allcoord_seg = get_seg_coord(get_rs_coord(get_lesion_coord()))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]


[78, 105, 89]
[50, 5, 44]
[81, 128, 106]
[76, 105, 89]
[52.38, 5.08, 44.57]
[79, 129, 106]


In [373]:
realspace_coordinates = np.dot(les_affine1328, [average_lesions[0][0],
                 average_lesions[0][1], 
                 average_lesions[0][2],
                 1])
average_lesions[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine_1328 = np.linalg.inv(seg_affine1328)
freesurfer_matrix = np.dot(inv_seg_affine_1328, [realspace_coordinates[0],
                 realspace_coordinates[1], 
                 realspace_coordinates[2],
                 1])
print average_lesions[0]; print realspace_coordinates; print freesurfer_matrix

[78, 105, 89]
[ 50.5039978    5.07699585  44.57289886   1.        ]
[  81.125       129.00000763  106.4375        1.        ]
